In [1]:
import sys 
sys.path.append('../')

from pathlib import Path
import pandas as pd
from utils.dataset import GraphDataset, data_split
from utils.transforms import *

root = Path('../EEGDataset/')
subjects = ['sub-01']

graph_data = GraphDataset(root, subjects, graph_type='corr_coef', reprocess=True)

Processing...
Done!


#### Visualize graph of random trial

In [2]:
from torch_geometric.data import Data

# Select a trial, load channel positions
sample = graph_data[10].clone()
edge_index = sample.edge_index
elec_coord_file = root/'electrode_coordinates.csv'
elec_coord = pd.read_csv(elec_coord_file, index_col=0)

labels = elec_coord.index.values
Xn = elec_coord['x'].values
Yn = elec_coord['y'].values
Zn = elec_coord['z'].values


Xe = []
Ye = []
Ze = []
for e in edge_index.T:
    Xe += [Xn[e[0]], Xn[e[1]], None]
    Ye += [Yn[e[0]], Yn[e[1]], None]
    Ze += [Zn[e[0]], Zn[e[1]], None]


import plotly.graph_objs as go

trace1 = go.Scatter3d(
    x=Xe, y=Ye, z=Ze,
    mode='lines',
    line=dict(color='rgb(0,0,0)', width=1),
    hoverinfo='none',
)

trace2 = go.Scatter3d(
    x=Xn, y=Yn, z=Zn,
    mode = 'markers',
    name = 'electrodes',
    marker = dict(
        symbol='circle',
        size=2,
        line=dict(color='rgb(50,50,50)',width=0.5),
    ),
    text=labels,
    hoverinfo='text',
)

axis=dict(
    showbackground=False,
    showline=False,
    showgrid=False,
    showticklabels=False,
    title='',
)

layout = go.Layout(
    scene = dict(
        xaxis=dict(axis),
        yaxis=dict(axis),
        zaxis=dict(axis),
    )
)

data = [trace1, trace2]
fig = go.Figure(data=data, layout=layout)
fig.show()

In [5]:
from utils.graph_utils import threshold_graph

edge_index = sample.edge_index 
edge_weights = sample.edge_weights 

print(sample.edge_index.shape)

torch.Size([2, 3250])


In [46]:
3200/160002

0.019999750003124962